In [ ]:
import numpy as np
from triplerecovery.blocks import make as make_blocks
from triplerecovery.utils import shuffle_under_seed
import triplerecovery as tr
from PIL import Image
import cv2

In [ ]:
# imarr=cv2.imread('./samples/art.png', cv2.IMREAD_GRAYSCALE)
imarr = np.array(cv2.cvtColor(
        cv2.VideoCapture('./samples/lena.gif').read()[1], cv2.COLOR_BGR2GRAY))
lookup = np.array([[0, 7, 13, 10],
                      [1, 6, 12, 11],
                      [4, 2, 9, 15],
                      [5, 3, 8, 14]], dtype=np.uint8)

In [ ]:
# making 16 main blocks
# size of single main block S= M/sqrt(16) X T=N/sqrt(16)
# e.g for lena the 512x512 the partner blocks size would be 16 blocks each with size (512x512)/(4x4) = 128x128
mainblock_shape = (int(imarr.shape[0]/4), int(imarr.shape[1]/4))
mainblocks = make_blocks(imarr, mainblock_shape)

# here the main blocks are like this index.
# 1st index is the block number.
# 2nd index is the channel (RGB) or 0 in Grey.
# 3rd and 4th are for indexing the block.

# Step3
# Ab har main block ko divide karo k 4x4 k block ban jain
# numberOfBlocks=(SxT)/(4x4)
# e.g 128x128/4x4 = 1024

avgblocks_shape = (4, 4)
averages = np.zeros((16, int((mainblock_shape[0]*mainblock_shape[1]) /
                              (avgblocks_shape[0]*avgblocks_shape[1]))), dtype=np.uint8)
# 4 indicatior A,B,C,D, 4 blocks of A, then the 4x4 Blocks which have count = (SxT)/(4x4), e.g 1024

for partner in lookup:  # A,B,C,D
    for id in partner:  # A1,A2,A3.....D4 etc
        averages[id] = make_blocks(mainblocks[id][0].copy(), avgblocks_shape,
                                    addChannel=False).mean((1, 2))

In [ ]:
averages.shape

In [ ]:
Image.fromarray(averages.reshape(128,128))

using new method

In [ ]:
newaverages=cv2.resize(imarr,None, fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)
newaverages.shape

In [ ]:
kk=make_blocks(newaverages, (32,32), addChannel=False)
kk.shape

In [ ]:
kk=kk.reshape(16,-1)

In [ ]:
Image.fromarray(kk.reshape(128,128))

In [ ]:
zoomx = 4  # according to the paper the zoom factor is 4
averages = cv2.resize(imarr, None, fx=1/zoomx, fy=1 /
                      zoomx, interpolation=cv2.INTER_AREA)

averages = make_blocks(
    averages, (imarr.shape[0]//zoomx, imarr.shape[1]//zoomx), addChannel=False).reshape(16, -1)
Image.fromarray(averages.reshape(128,128))

In [ ]:
rc=tr.bits.recovery.make(imarr,lookup,key="")

In [ ]:
rc.shape

In [ ]:
eimarr=tr.bits.recovery.embed(imarr, rc)
eimarr.shape

In [ ]:
Image.fromarray(eimarr)

In [ ]:
ext=tr.bits.recovery.extract(eimarr, key="")
ext.shape

In [ ]:
ext

from recovery bits to the orignal image

In [ ]:
aut = tr.authenticate(imarr)


In [ ]:
aut.tempred.shape

In [ ]:
tempred=~aut.tempred

In [ ]:
tempred=np.zeros(tempred.shape,dtype=bool)

In [ ]:
averages=np.zeros((16,8192),  dtype=np.uint8)

In [ ]:
mask=aut.maskarr.copy()

In [ ]:
mask[0][0]=123

In [ ]:
mask.astype(bool)

In [ ]:
lookup

In [ ]:
lst=np.zeros((16,4), dtype=np.uint8)
for idx,partner in enumerate(lookup):
  for id in partner:
    f=[id]
    f.extend([i for i in partner if i!=id])
    lst[id]=f

In [ ]:
lookup

In [ ]:
lst

In [ ]:
recovery_bits=ext

In [ ]:
for partner in range(lookup.shape[0]):  # A,B,C,D
  for id in range(lookup.shape[1]):  # A1,A2,A3.....D4 etc
    # get the partner block of this id also the id it sits
        partner_block = -1, -1
        for i in range(lookup[partner].shape[0]):
            # if this id isn't tempred, we can stop
            if i != id and not np.any(tempred[lookup[partner, i]]):
                partner_block = partner, i
                break

        if partner_block == (-1, -1):
            print(
                "Could not find partner block for {} {}".format(partner, id))
            continue
        
        # get recovery bits of this id
        # get index of this id in the partner block
        idx = id
        if partner_block[1] < id:
            idx -= 1
        print((partner,id),partner_block,lookup[partner_block])
        print("start: ", idx*recovery_bits.shape[1]//3," end: ",idx*recovery_bits.shape[1]//3+recovery_bits.shape[1]//3)
        averages[lookup[(partner,id)]]=recovery_bits[lookup[partner_block],
                                                          idx*recovery_bits.shape[1]//3:idx*recovery_bits.shape[1]//3+recovery_bits.shape[1]//3]
        

In [ ]:
recovery_bits[15]

In [ ]:
partner_block

In [ ]:
averages[15]

In [ ]:
averages.shape

In [ ]:
averages2=np.packbits(averages, axis=1)

In [ ]:
aut.maskarr

In [ ]:
averages2

In [ ]:
Image.fromarray(averages2.reshape((128,128),-1))

In [ ]:
recreated=cv2.resize(averages2.reshape(128,128),None, fx=4,fy=4,interpolation=cv2.INTER_LANCZOS4)

In [ ]:
Image.fromarray(recreated)